In [ ]:
# download the data, data should be moved to "data/"
# https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-California.json.gz
# https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/meta-California.json.gz 

In [ ]:
# all necesary imports
import numpy as np
import defaultdict
import pandas as pd

# Introduction

TODO: Advaith

# Dataset

TODO: Advaith

### Data Cleaning

TODO Ketki

### Exploratory Data Analysis (EDA)

TODO: Ketki

# Predicitive Task

TODO: Advaith

### Model

TODO: Arnav

# Results

TODO: Advaith, Arnav, Ketki

# Conclusion

TODO: Advaith

# Literatute/Citation Work

TODO: Advaith